In [1]:
import matplotlib
import numpy as np
import math
import pandas as pd
import matplotlib
import matplotlib.pylab as plt
import random
from sklearn import preprocessing
import torch
from torch.nn import Linear
from torch import Tensor
from torch.nn import MSELoss
from torch.optim import SGD, Adam, RMSprop
from torch.autograd import Variable, grad
from torch.utils.data.sampler import SubsetRandomSampler,WeightedRandomSampler

from mpl_toolkits.mplot3d import Axes3D

import sys
import pandas as pd

manualSeed = 1 # random.randint(1, 10000) # Set the desired seed to reproduce the results
torch.manual_seed(manualSeed)

np.random.seed(1)

# Dataset Definition

Dataset includes 5 input values (aspect ratio, dimensionless frequency, velocity contrast, frequency component), and real and imaginary part of time-series as output. 

In [2]:
def CreateDataset(PATH):
    
    Xp_1 = np.load('Data/Xp.npy')
    Xp_2 = np.load('Data/Xp_1.npy')
    Xp = np.concatenate((Xp_1,Xp_2))
    
    Yp_1 = np.load('Data/Yp.npy')
    Yp_2 = np.load('Data/Yp_1.npy')
    Yp = np.concatenate((Yp_1,Yp_2))
    
    Data = np.column_stack((Xp,Yp))
    data_size = len(Data)
    r = np.random.permutation(data_size)
    Data = Data[r,:]
    start = 596
    step = 2
    border = 895
    
    
    Xp = Data[:110000,start:895]
    Yp = Data[:110000,895::step]
    
    
    Xp = preprocessing.scale(Xp)
    
    Yp = preprocessing.scale(Yp)
    
    Xp_test = Data[110000:,start:895]
    Yp_test = Data[110000:,895::step]
    
    
    return Xp,Yp, Xp_test, Yp_test



# Neural Network Achietecture


In [3]:
class Reg_model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Defining the neural network
        self.relu = torch.nn.ReLU()
        m = 1
        # Layers 
        self.fc0  = Linear(299,299*m)
        self.fc1  = Linear(299*m,299*m)

        # resnet - block 1
        self.rn1_fc1  = Linear(299*m,299*m)
        self.rn1_fc2  = Linear(299*m,299*m)
        self.rn1_fc3  = Linear(299*m,299*m)

        # resnet - block 2 
        self.rn2_fc1  = Linear(299*m,299*m)
        self.rn2_fc2  = Linear(299*m,299*m)
        self.rn2_fc3  = Linear(299*m,299*m)

        # resnet - block 3
        self.rn3_fc1  = Linear(299*m,299*m)
        self.rn3_fc2  = Linear(299*m,299*m)
        self.rn3_fc3  = Linear(299*m,299*m)


        # Output structure
        self.fc8  = Linear(299*m,299)
        self.fc9  = Linear(299,125)

    def forward(self, x):
        
        x   = self.relu(self.fc0(x))
        x   = self.relu(self.fc1(x))

        # Resnet - Block 1
        x0  = x
        x   = self.relu(self.rn1_fc1(x))
        x   = self.relu(self.rn1_fc3(x) + self.rn1_fc2(x0))


        # Resnet - Block 2
        x0  = x
        x   = self.relu(self.rn2_fc1(x))
        x   = self.relu(self.rn2_fc3(x)+self.rn2_fc2(x0))

#         # Resnet - Block 3
        x0  = x
        x   = self.relu(self.rn3_fc1(x))
        x   = self.relu(self.rn3_fc3(x)+self.rn3_fc2(x0))



        # Joining two blocks
        x     = self.relu(self.fc8(x))
        x   = self.fc9(x)
        return x

In [4]:
class NumpyDataset(torch.utils.data.Dataset):

    def __init__(self, data, target, transform=None):
        # Creating identical pairs
        self.data    = Variable(Tensor(data))
        self.target  = Variable(Tensor(target))

    def send_device(self,device):
        self.data    = self.data.to(device)
        self.target  = self.target.to(device)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        return x, y, index

    def __len__(self):
        return self.data.shape[0]

def init_weights(m):
    if type(m) == torch.nn.Linear:
        stdv = (1. / math.sqrt(m.weight.size(1))/1.)*2
        m.weight.data.uniform_(-stdv,stdv)
        m.bias.data.uniform_(-stdv,stdv)

def Loss_fun(Yobs,Ypred):

        loss_2 = loss_funcl2(Ypred, Yobs)
        loss_1 = loss_funcl1(Ypred, Yobs)/10
    
#         return loss_1+loss_2
        return loss_2

# def eval(model, val_loader, device):
#     model.eval()
#     l2 = 0
#     l1 = 0
    
#     with torch.no_grad():
#         for batch_idx, (data, target) in enumerate(val_loader):
#             data, target = Variable(data).to(device), Variable(target).to(device)
#             output = model(data)
#             l2 += loss_funcl2(output, target)
#             l1 += loss_funcl1(output, target)
            
            
            
#     l1 /= len(val_loader.dataset) / bs
#     l2 /= len(val_loader.dataset) / bs
    
# #     model.train()
#     return l2.detach()

In [5]:
class Model():
    def __init__(self, network, optimizer,scheduler, model_path, model_name):
        self.network    = network
        self.optimizer  = optimizer
        self.model_path = model_path
        self.model_name = model_name
        self.scheduler  = scheduler

        self.total_train_loss = []
        self.total_val_loss   = []
        
    def train(self, dataset,n_epochs):
        from torch.autograd import Variable
        import time

        len_dataset         = len(dataset)
        n_batches           = int(len(dataset)/btsize + 1)
        training_start_time = time.time()

        # Sending the data to CPU
        dataset.send_device(device) 

        # --------- Splitting the dataset into training and validation -------
        indices            = list(range(int(len_dataset)))
        validation_idx     = indices[:int(len_dataset*(validation_per/100))]  #np.random.choice(indices, size=int(len_dataset*(validation_per/100)), replace=False)
        train_idx          = list(set(indices) - set(validation_idx))
        validation_sampler = SubsetRandomSampler(validation_idx)
        train_sampler      = SubsetRandomSampler(train_idx)

        train_loader       = torch.utils.data.DataLoader(
            dataset,
            batch_size=btsize,
            sampler=train_sampler,
            )    
        validation_loader  = torch.utils.data.DataLoader(
            dataset,
            batch_size=btsize,
            sampler=validation_sampler,
        )    

        for epoch in range(n_epochs):
            print_every           = 1
            start_time            = time.time()
            running_sample_count  = 0

            total_train_loss      = 0
            total_val_loss        = 0


            for i, data in enumerate(train_loader, 0):
                #print('----------------- Epoch {} - Batch {} --------------------'.format(epoch,i))
                
                # Get inputs/outputs and wrap in variable object
                inputs, labels, indexbatch = data
                # Making sure input and labels are floats
                inputs.float()
                labels.float()
                
                inputs.requires_grad_()

                # Forward pass
                pred_labels = self.network(inputs)
                loss_ = Loss_fun(labels,pred_labels)

    
                # Update parameters
                loss_.backward()
                self.optimizer.step()
                self.optimizer.zero_grad()



            # ----- Determining the Training Loss -----
            for i, data_train in enumerate(train_loader, 0):
                inputs_train, labels_train, indexbatch_train = data_train
                inputs_train.requires_grad_()
#                 outputs_train,pred_labels_train = self.network(inputs_train)
                pred_labels_train = self.network(inputs_train)
                train_loss              = Loss_fun(labels_train,pred_labels_train)
                total_train_loss           += train_loss.item()
#                 del inputs_train, labels_train, indexbatch_train, pred_labels_train, train_loss, wv


            # ----- Determining the Validation Loss -----
            for i, data_val in enumerate(validation_loader, 0):
                inputs_val, labels_val, indexbatch_val = data_val
                inputs_val.requires_grad_()
                pred_labels_val = self.network(inputs_val)
                val_loss                 = Loss_fun(labels_val,pred_labels_val)
                total_val_loss             += val_loss.item()




            # Creating a running loss for both training and validation data
            total_val_loss   /= len(validation_loader)
            total_train_loss /= len(train_loader)
            self.total_train_loss.append(total_train_loss)
            self.total_val_loss.append(total_val_loss)
            self.scheduler.step(total_val_loss)

            if (epoch+1) % 10 == 1:
                with torch.no_grad():
                    print("Epoch = {} -- Training loss = {:.4e} -- Validation loss = {:.4e}".format(epoch+1,total_train_loss,total_val_loss))

            if (epoch+1) % 100 == 1:
                with torch.no_grad():
                    torch.save({
                        'epoch':epoch,
                        'model_state_dict': self.network.state_dict(),
                        'optimizer_state_dict': self.optimizer.state_dict(),
                        'train_loss': self.total_train_loss,
                        'val_loss': self.total_val_loss,
                        }, '{}/{}_{}_{}.pt'.format(self.model_path, self.model_name,str(epoch).zfill(5),total_val_loss))

In [6]:
num_epochs         = 5000
validation_per     = 10
btsize             = 256


# -------------------------------------------------------------------
PATH                                 = './OutputModels'
Xp,Yp, Xp_test, Yp_test              = CreateDataset(PATH)
dataset                              = NumpyDataset(Xp,Yp)

# -------------------------------------------------------------------
# device             = torch.device("cuda:{}".format(GPUid))
device = torch.device("cpu")
torch.set_num_threads(10)

# --------- Defining the neural network -------
net = Reg_model()
net.apply(init_weights)
net.float()
net.to(device)
loss_funcl2 = torch.nn.MSELoss(reduction='mean')
loss_funcl1 = torch.nn.L1Loss(reduction='mean')

def count_parameters(net):
    return sum(p.numel() for p in net.parameters() if p.requires_grad)
print(count_parameters(net))

(110000, 299)
(110000, 125)
1113900


In [1]:
## --------- Defining optimizer -------
optimizer  = torch.optim.Adam(net.parameters(),lr=1e-4)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

# --------- Training Model -------
model = Model(net, optimizer,scheduler,model_path=PATH,model_name='1D_inversion')
model.train(dataset,num_epochs)